In [188]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [189]:
matches_df = pd.read_csv('Dataset/matches.csv')
match_details = pd.read_csv('Dataset/deliveries.csv')

In [190]:
# taking a fraction of data due to its large size
matches_df = matches_df[:195]
match_details = match_details[:100273]

In [191]:
matches_df.head()

,id,year,date,winner,result,win_by_runs,win_by_wickets,player_of_match,team1,team2,toss_winner,toss_decision,umpire1,umpire2,venue
0,1,2017,2017-01-13,Australia,normal,92.0,NaN,MS Wade,Australia,Pakistan,Australia,bat,MD Martell,C Shamshuddin,"Brisbane Cricket Ground, Woolloongabba"
1,2,2017,2017-01-15,Pakistan,normal,NaN,6.0,Mohammad Hafeez,Australia,Pakistan,Australia,bat,CB Gaffaney,P Wilson,Melbourne Cricket Ground
2,3,2017,2017-01-19,Australia,normal,NaN,7.0,SPD Smith,Australia,Pakistan,Australia,field,SD Fry,C Shamshuddin,Western Australia Cricket Association Ground
3,4,2017,2017-01-22,Australia,normal,86.0,NaN,DA Warner,Australia,Pakistan,Australia,bat,CB Gaffaney,MD Martell,Sydney Cricket Ground
4,5,2017,2017-01-26,Australia,normal,57.0,NaN,DA Warner,Australia,Pakistan,Australia,bat,SD Fry,C Shamshuddin,Adelaide Oval


In [192]:
match_details.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,bye_runs,legbye_runs,noball_runs,batsman_runs,extra_runs,total,player_dismissed,dismissal_kind,fielder
0,1,1,Australia,Pakistan,1,1,DA Warner,TM Head,Mohammad Amir,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Australia,Pakistan,1,2,DA Warner,TM Head,Mohammad Amir,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Australia,Pakistan,1,3,DA Warner,TM Head,Mohammad Amir,0,0,0,0,0,0,0,NaN,NaN,NaN
3,1,1,Australia,Pakistan,1,4,DA Warner,TM Head,Mohammad Amir,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Australia,Pakistan,1,5,DA Warner,TM Head,Mohammad Amir,1,0,0,0,0,1,1,NaN,NaN,NaN


In [193]:
len(match_details)

100273

In [100]:
count=0
for i in range(len(match_details)):
    if  match_details.iloc[i]['wide_runs']!=0 or match_details.iloc[i]['noball_runs']!=0:
        count+=1
v=count/len(match_details)
print("Estimation of wide/no ball = {}".format(v))

Estimation of wide/no ball = 0.025011718009833154


### We got our Wide/No ball probability  v = 0.0250
![123](images/4.jpg)

# 100272 194

In [194]:
print("Total Matches:{}".format(len(matches_df)))
print("total Teams: {}".format(len(set(matches_df.team1))))
print("Total number of Batsmen: {}".format(len(set(match_details.batsman))))
print("Total number of Bowlers: {}".format(len(set(match_details.bowler))))
print("Estimation of wide/no ball = {}".format(v))
print("Total balls Bowled: {}".format(len(match_details)))

Total Matches:195
total Teams: 15
Total number of Batsmen: 528
Total number of Bowlers: 386
Estimation of wide/no ball = 0.025011718009833154
Total balls Bowled: 100273


In [195]:
# feature engineering
drop_col_list = ['non_striker','wide_runs', 'bye_runs',
       'legbye_runs', 'noball_runs', 'batsman_runs', 'extra_runs','dismissal_kind', 'fielder']
match_details = match_details.drop(drop_col_list,axis=1)
match_details.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,bowler,total,player_dismissed
0,1,1,Australia,Pakistan,1,1,DA Warner,Mohammad Amir,0,NaN
1,1,1,Australia,Pakistan,1,2,DA Warner,Mohammad Amir,0,NaN
2,1,1,Australia,Pakistan,1,3,DA Warner,Mohammad Amir,0,NaN
3,1,1,Australia,Pakistan,1,4,DA Warner,Mohammad Amir,0,NaN
4,1,1,Australia,Pakistan,1,5,DA Warner,Mohammad Amir,1,NaN


In [196]:
# Calculating the Score for each ball
Score=[]
total_run = 0
temp=1
for i,runs in enumerate(match_details.total):
    if temp != match_details.iloc[i]["inning"]:
        total_run=0
        temp = match_details.iloc[i]["inning"]
    total_run += runs
    Score.append(total_run)
print(len(Score))

100273


In [197]:
# Check Is out or not
is_wicket=[]
out=0
for i,wicket in enumerate(match_details.player_dismissed):
    if  pd.isnull(wicket) == False:
        out=1
    else:
        out=0
    is_wicket.append(out)
len(is_wicket)

100273

In [198]:
#Calculation for Wickets taken
Wickets=[]
total_wickets=0
temp=1
for i,wicket in enumerate(match_details.player_dismissed):
    if  pd.isnull(wicket) == False:
        total_wickets+=1
    if temp != match_details.iloc[i]["inning"]:
            total_wickets=0
            temp = match_details.iloc[i]["inning"]
    Wickets.append(total_wickets)
print(len(Wickets))        

100273


In [199]:
#Calculation for Balls_Bowled
Balls_Bowled=[]
total_balls=0
temp=1
for i,ball in enumerate(match_details.ball):
    if ball <=6:
        if temp != match_details.iloc[i]["inning"]:
            total_balls=0
            temp = match_details.iloc[i]["inning"]
        total_balls+=1
    Balls_Bowled.append(total_balls)
len(Balls_Bowled)

100273

In [200]:
### Adding the new features into the dataset
match_details.rename(columns = {'total':'runs_accquired'}, inplace = True)
match_details = match_details.drop('player_dismissed',axis=1)
match_details['Score'] = Score
match_details['Wickets'] = Wickets
match_details['Balls_Bowled'] = Balls_Bowled
match_details['Is_OUT'] = is_wicket

In [201]:
match_details.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'bowler', 'runs_accquired', 'Score', 'Wickets',
       'Balls_Bowled', 'Is_OUT'],
      dtype='object')

In [202]:
match_details = match_details[['match_id', 'inning', 'batting_team', 'bowling_team', 'over','Balls_Bowled','ball','batsman', 'bowler', 'runs_accquired', 'Score', 'Wickets','Is_OUT']]

### Now our dataset has all the feature to predict the outcome
![as](images/3.jpg)

In [203]:
match_details.head()

,match_id,inning,batting_team,bowling_team,over,Balls_Bowled,ball,batsman,bowler,runs_accquired,Score,Wickets,Is_OUT
0,1,1,Australia,Pakistan,1,1,1,DA Warner,Mohammad Amir,0,0,0,0
1,1,1,Australia,Pakistan,1,2,2,DA Warner,Mohammad Amir,0,0,0,0
2,1,1,Australia,Pakistan,1,3,3,DA Warner,Mohammad Amir,0,0,0,0
3,1,1,Australia,Pakistan,1,4,4,DA Warner,Mohammad Amir,0,0,0,0
4,1,1,Australia,Pakistan,1,5,5,DA Warner,Mohammad Amir,1,1,0,0


In [204]:
# Label Encoding of categorical values
from sklearn import preprocessing
le_team = preprocessing.LabelEncoder()
le_batsman = preprocessing.LabelEncoder()
le_bowler = preprocessing.LabelEncoder()

match_details.batting_team = le_team.fit_transform(match_details.batting_team)
match_details.bowling_team = le_team.fit_transform(match_details.bowling_team)
match_details.batsman = le_batsman.fit_transform(match_details.batsman)
match_details.bowler = le_batsman.fit_transform(match_details.bowler)

In [205]:
# After Label Encoding
match_details.head()

,match_id,inning,batting_team,bowling_team,over,Balls_Bowled,ball,batsman,bowler,runs_accquired,Score,Wickets,Is_OUT
0,1,1,1,8,1,1,1,114,239,0,0,0,0
1,1,1,1,8,1,2,2,114,239,0,0,0,0
2,1,1,1,8,1,3,3,114,239,0,0,0,0
3,1,1,1,8,1,4,4,114,239,0,0,0,0
4,1,1,1,8,1,5,5,114,239,1,1,0,0


### At this point our features are ready, time to add the target variable in our dataset

![abc](images/2.jpg)

In [206]:
Target_var = []
target =0
for i in range(len(match_details)):
    if match_details.iloc[i]['Is_OUT']==1:
        target = 1
    elif match_details.iloc[i]['runs_accquired']==0:
        target = 2
    elif match_details.iloc[i]['runs_accquired']==1:
        target = 3
    elif match_details.iloc[i]['runs_accquired']==2:
        target = 4
    elif match_details.iloc[i]['runs_accquired']==3:
        target = 5
    elif match_details.iloc[i]['runs_accquired']==4:
        target = 6
    elif match_details.iloc[i]['runs_accquired']==6:
        target = 7
    Target_var.append(target)
len(Target_var)

100273

In [207]:
match_details['Target'] = Target_var

In [208]:
match_details.head()

,match_id,inning,batting_team,bowling_team,over,Balls_Bowled,ball,batsman,bowler,runs_accquired,Score,Wickets,Is_OUT,Target
0,1,1,1,8,1,1,1,114,239,0,0,0,0,2
1,1,1,1,8,1,2,2,114,239,0,0,0,0,2
2,1,1,1,8,1,3,3,114,239,0,0,0,0,2
3,1,1,1,8,1,4,4,114,239,0,0,0,0,2
4,1,1,1,8,1,5,5,114,239,1,1,0,0,3


## Checkpoint:

In [209]:
match_details.to_csv('Processed_dataset.csv',index=False)

## Finally our dataset is ready for modeling

* In modeling , Given some features we will generate the outcome using the equation below:
![asx](images/5.jpg)
* But to get the result of Eq 3 we need another equation given below:
![ada](images/6.jpg)
* We can get unknown parameters for equation 6 from below:
![qwe](images/7.jpg)
* In short:
![wrte](images/8.jpg)

In [398]:
# generating the unkown parameters before the distributions
del_1 = np.random.uniform()
del_2 = np.random.uniform()

tow = np.random.gamma(1.0,1.0)
sigma = np.random.gamma(1.0,1.0)

u_i = (tow**2)*np.random.randn(528) #batsman
u_j = (tow**2)*np.random.randn(386) #bowler

![234](images/11.jpg)

In [399]:
def generate_alk():
    alk = []
    for i in range(9):
        ak=[]
        for j in range(6):
            if len(ak)==0:
                ak.append(np.random.randn())
            else:
                temp = np.random.randn()
                while(ak[-1]>temp):
                    temp= np.random.randn()
                ak.append(temp)
        alk.append(ak)
    return alk

In [400]:
a_lk = generate_alk()
a_lk = np.array(a_lk)
a_lk = (sigma**2)*a_lk

In [401]:
a_lk

array([[-0.62892396,  0.23881564,  1.00242203,  1.42205023,  1.55806653,
         1.71149593],
       [ 0.81407072,  0.9408261 ,  1.08219004,  1.15414047,  1.20558939,
         1.60984593],
       [ 0.53582691,  0.75133601,  0.96270398,  0.98028865,  1.28692581,
         1.95290497],
       [ 0.42817781,  0.51970156,  1.27341475,  1.40966968,  1.43030072,
         1.4551182 ],
       [-0.50016836, -0.39491516,  0.4152678 ,  0.53365398,  0.89078499,
         1.17615392],
       [ 0.61208451,  0.78137341,  1.36403914,  1.3685419 ,  1.38758707,
         1.5421192 ],
       [ 0.27957286,  0.55268545,  2.02965436,  2.34074399,  2.39608686,
         2.6944311 ],
       [ 0.45911305,  0.90716723,  1.19455916,  1.37018114,  1.43402154,
         1.43989578],
       [ 0.49930394,  0.85086306,  0.87713535,  1.01640984,  1.09255721,
         1.29955012]])

## implementing the situation
![rrg](images/9.jpg)

In [361]:
def Situation(over,wicket_loss):
    if 1<=over<=15 and 0<=wicket_loss<=3:
        return 1
    if 16<=over<=35 and 0<=wicket_loss<=3:
        return 2
    if 36<=over<=50 and 0<=wicket_loss<=3:
        return 3
    if 1<=over<=15 and 4<=wicket_loss<=6:
        return 4
    if 16<=over<=35 and 4<=wicket_loss<=6:
        return 5
    if 36<=over<=50 and 4<=wicket_loss<=6:
        return 6
    if 1<=over<=15 and 7<=wicket_loss<=9:
        return 7
    if 16<=over<=35 and 7<=wicket_loss<=9:
        return 8
    elif 36<=over<=50 and 7<=wicket_loss<=9:
        return 9

![ada](images/6.jpg)
![ada1](images/10.jpg)

In [440]:
def Predicted_P_ijwbk(i,j,k,l):
    new_del = 0
    if l <=3:
        new_del=0
    elif 3<l<=6:
        new_del=del_1
    elif 6<l<=9:
        new_del=del_1+del_2
    
    #print(term1,term2)
    if k==1 or k==7:
        result = 0.01
    else:
        term1 = a_lk[l-1][k-1] -u_i[i] - new_del + u_j[j] #indexing of python 
        term2 = a_lk[l-1][k-2] -u_i[i] - new_del + u_j[j]
        result = np.random.logistic(term1) - np.random.logistic(term2)
    return result

In [451]:
Predicted_P_ijwbk(1,1,5,1)

1.5628461861397416

## So, We Finally got Our... Prob(Xb=k)
![asx](images/5.jpg)

In [ ]:
#### Check Predicted_P_ijwbk(i,j,k,l) for 1 and 7

![rf](images/12.jpg)

In [368]:
match_details.Target.value_counts()

2    50886
3    32760
6     7603
4     4736
1     2699
7     1114
5      475
Name: Target, dtype: int64

In [371]:
target_freq = np.array([2699,50886,32760,4736,475,7603,1114])/len(match_details)

In [452]:
target_freq

array([0.02691652, 0.50747459, 0.32670809, 0.04723106, 0.00473707,
       0.075823  , 0.01110967])

In [387]:
## Testing out algorithm on a Match
match1 = match_details[match_details.match_id == 1]
match1_1st = match1[match1.inning==1]

In [388]:
match1_1st

,match_id,inning,batting_team,bowling_team,over,Balls_Bowled,ball,batsman,bowler,runs_accquired,Score,Wickets,Is_OUT,Target
0,1,1,1,8,1,1,1,114,239,0,0,0,0,2
1,1,1,1,8,1,2,2,114,239,0,0,0,0,2
2,1,1,1,8,1,3,3,114,239,0,0,0,0,2
3,1,1,1,8,1,4,4,114,239,0,0,0,0,2
4,1,1,1,8,1,5,5,114,239,1,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,1,1,1,8,50,296,2,324,130,4,265,9,0,6
306,1,1,1,8,50,297,3,324,130,0,265,9,0,2
307,1,1,1,8,50,298,4,324,130,0,265,9,0,2
308,1,1,1,8,50,299,5,324,130,2,267,9,0,4


### Observation on the data
* Australia is the batting in the 1st innings
* Pakisthan is bowling
* Opening Batsmen are DA Warner and TM Head
* 1st Bowler is Mohammad Amir

### Important Insight
There are **310 rows** in this dataset, A one innings have only **300 balls(50 overs)**<br>
So, **the extra 10 balls must be wide or no balls**<br>

In [389]:
# Overall Overview of the match
matches_df.iloc[0]

id                                                      1
year                                                 2017
date                                           2017-01-13
winner                                          Australia
result                                             normal
win_by_runs                                            92
win_by_wickets                                        NaN
player_of_match                                   MS Wade
team1                                           Australia
team2                                            Pakistan
toss_winner                                     Australia
toss_decision                                         bat
umpire1                                        MD Martell
umpire2                                     C Shamshuddin
venue              Brisbane Cricket Ground, Woolloongabba
Name: 0, dtype: object

## Algorithm:
<img src="images/13.jpg" width="600" height="600">

In [391]:
print(v)
print(target_freq)

0.025011718009833154
[0.02691652 0.50747459 0.32670809 0.04723106 0.00473707 0.075823
 0.01110967]


In [442]:
for dat in range(len(match1_1st)):
    u = np.random.uniform()
    if u < v:
        print("WideB:")
        print(target_freq[1],target_freq[2],target_freq[3],target_freq[4],target_freq[5],target_freq[6])
    else:
        #Xb
        l = Situation(match1_1st.iloc[dat]["over"],match1_1st.iloc[dat]["Wickets"])
        i = match1_1st.iloc[dat]["batsman"]
        j = match1_1st.iloc[dat]["bowler"]
        print("Ball:")
        print(Predicted_P_ijwbk(i,j,1,l),Predicted_P_ijwbk(i,j,2,l),Predicted_P_ijwbk(i,j,3,l),Predicted_P_ijwbk(i,j,4,l),Predicted_P_ijwbk(i,j,5,l),Predicted_P_ijwbk(i,j,6,l),Predicted_P_ijwbk(i,j,7,l))


Ball:
0.01 1.2647779156301038 0.24779103272327774 1.2056124968616293 0.09030919033277995 6.193717705730348 0.01
Ball:
0.01 1.6403408394407775 2.7019858106221033 4.691919343571573 1.4696186504457576 0.92265740415376 0.01
Ball:
0.01 2.517191114989044 5.314143632827053 1.4843050384287118 1.7075869576059093 3.2684142862746572 0.01
Ball:
0.01 2.677643635255734 3.4132031796321614 0.4972500213594194 1.6513927645413589 3.1204032045712546 0.01
Ball:
0.01 6.161701354540779 1.4631691552025465 2.8824184448105257 0.8062636264123535 5.866418209876107 0.01
Ball:
0.01 0.7773596220030153 1.0259158451674186 3.2498136236506268 0.9638087964033333 0.024678739522740578 0.01
Ball:
0.01 1.6324297653518287 2.0458858880204596 0.6994915714004746 0.5855646113157165 2.10032199530983 0.01
Ball:
0.01 1.1042023145430382 3.1045282405134866 0.6151778797797829 1.5356194296998154 3.345915782389242 0.01
Ball:
0.01 2.0117929167953648 3.326397420185096 0.6014864115764205 0.8871341942371398 0.8763858876219492 0.01
Ball:
0.01

0.01 3.2114658286387625 2.7503314152167455 0.09202159849302327 3.424764256609845 4.459685009222273 0.01
Ball:
0.01 1.9998079998223954 1.4358715930572683 0.06578309745367505 0.20552026222430836 1.9456979623064754 0.01
Ball:
0.01 0.44802432054548724 2.1019679310966297 3.762675158328787 5.841205556847166 0.9493138393719152 0.01
Ball:
0.01 1.9014651115016639 0.026529622582936785 4.0864592611383035 0.06118527981931382 2.5767632428745086 0.01
Ball:
0.01 1.9586837804178407 2.8050084879690633 4.228409880352932 2.558391752491332 2.0157359763945966 0.01
Ball:
0.01 3.0207549461222962 2.59254847142119 1.6997368373092168 2.402665363385912 1.6401550282503587 0.01
Ball:
0.01 2.0959017461133036 1.6454266477150419 4.238320414667973 0.4081241147153163 1.0118843861329592 0.01
Ball:
0.01 2.070825116399801 1.4110478527171808 0.43364299849182597 0.3045207869909192 1.2196875901141508 0.01
Ball:
0.01 0.3155487542988933 3.566868983790518 0.8632476826790038 0.8856664467715608 4.235894053061482 0.01
Ball:
0.01 2

0.01 0.21177648345802014 3.7680445817682298 0.7105543700842276 0.9298343643219407 1.7948655640485258 0.01
Ball:
0.01 3.9523761265392743 2.4388260919388234 0.7174830435220139 0.965165015528938 2.4677578614095346 0.01
Ball:
0.01 3.8092949500755307 0.13028001806620204 1.678771130888845 1.3315742167066182 0.4844036477941667 0.01
Ball:
0.01 0.1968455094493295 1.9525070388051728 0.22397401775462739 1.0649198321018642 0.35234792379535596 0.01
Ball:
0.01 3.8654017474535043 0.6587628337053963 0.6347965679326559 1.1342094568892054 1.76359660801886 0.01
Ball:
0.01 0.45741729919873997 0.6159201630240035 1.8746517569124197 3.3426864814492445 1.6978184833901135 0.01
Ball:
0.01 2.738664280906915 2.1201851337761033 0.7165845019746169 0.1181968916921805 2.408622405800272 0.01
Ball:
0.01 0.5635712330570073 0.06073364505757825 3.3641239101926854 3.7128655148406904 2.5818114716308846 0.01
Ball:
0.01 1.893976250745578 1.8187793434147261 1.320372416767985 4.168409990226598 1.5197858423034445 0.01
Ball:
0.01